<img src="media/onda_logo.jpg" alt="logo onda" height="200" width="200">

# AOI search via a Jupyter notebook

In [1]:
import os
from modules import aoi, buttons
from ipywidgets import interact

## Draw a polygon on the map! 
- Select an area of interest (AOI) choosing a shape among the buttons on the left side
- Only one polygon per time is supported by this version
- *!!* Do not scroll world window and draw your AOI in a world copy. This will generate non valid coordinates for the query ([Issue #96](https://github.com/jupyter-widgets/ipyleaflet/issues/96))

In [2]:
m,dc = aoi.define_map()
def handle_draw(self, action, geo_json):
    print("Action: %s"%action)
    print(geo_json["geometry"])
    buttons.save_aoi(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)
display(m)

Map(center=[40.853294, 14.305573], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Action: created
{'type': 'Polygon', 'coordinates': [[[6.272026, 41.158805], [6.272026, 46.540727], [14.581717, 46.540727], [14.581717, 41.158805], [6.272026, 41.158805]]]}

Selection saved


## Set some filters!
- Remember to submit each field (even if not set).

In [3]:
buttons._filters_()

DatePicker(value=None, description='Sensed from:')

DatePicker(value=None, description='Sensed to:')

Button(description='Submit sensing range', style=ButtonStyle())

Select(options={'All': ['-'], 'Sentinel-1': ['All S1', 'SLC', 'GRD', 'OCN', 'RAW'], 'Sentinel-2': ['All S2', '…

Button(description='Submit mission', style=ButtonStyle())

Sensing range from 2020-02-01 to 2020-03-12


Dropdown(description='Product type:', layout=Layout(width='30%'), options=('All S5P', 'L1B_IR_SIR', 'L1B_IR_UV…

Button(description='Submit product type', style=ButtonStyle())

Mission: Sentinel-5 Precursor && Product Type: L2__NO2___


In [4]:
polysel = buttons.read_aoi()
item = buttons.read_mp()
sensing = buttons.read_sen()
n = aoi.count(polysel,item,sensing)
df = aoi.search(n,polysel,item,sensing)

Searching for products...
https://catalogue.onda-dias.eu/dias-catalogue/Products/$count?$search=%22(%20(%20(%20platformName:Sentinel-5 Precursor%20AND%20productType:L2__NO2___%20)%20)%20AND%20(%20(%20beginPosition:[2020-02-01T00:00:00.000Z%20TO%202020-03-12T23:59:59.999Z]%20AND%20endPosition:[2020-02-01T00:00:00.000Z%20TO%202020-03-12T23:59:59.999Z]%20)%20)%20)%20AND%20footprint:%22Intersects(POLYGON((6.272026%2041.158805,6.272026%2046.540727,14.581717%2046.540727,14.581717%2041.158805,6.272026%2041.158805)))%22%22
52 products found


In [5]:
@interact
def _iFP_(product=[f for f in df.iloc[:,1]]): 
    keys = df.index[df['name'] == product].tolist()
    m = aoi.update_Map(keys,polysel,df)
    display(m)
    display(df.loc[keys])
    aoi.warning(df.loc[keys].iloc[:,6].any())
    print("Find item at: %s"%os.path.join(str(df.loc[keys].iloc[:,2].any()),str(df.loc[keys].iloc[:,1].any())))

interactive(children=(Dropdown(description='product', options=('S5P_OFFL_L2__NO2____20200201T094106_20200201T1…

## Save your selections!

- Choose products you wish to collect and submit each of them. This will produce a dat set named *saved_query.csv* in the *output* folder. 
- *!!* Warning. Re-running this cell will erase the data set. Select and submit items in the current run.

In [ ]:
sel = buttons.select_product(df)
sel.display_widgets() 

## Copy AOI dataframe in a destination folder
You may find useful to copy this dataframe into a destination folder. Specify the filename (saved_query.json or polygons.json) and the destination path. By default path is set to outputs folder.

In [ ]:
file = "outputs/list_remote.txt" # dataframe containing metadata 
dest = os.path.join(os.getcwd(),"S2") # destination path here
aoi.copy2(os.path.join(os.getcwd(),file),dest) 

In [ ]:
file = "outputs/polygon.json" # selected polygon
dest = os.path.join(os.getcwd(),"S5P") # destination path here
aoi.copy2(os.path.join(os.getcwd(),file),dest) 